In [1]:
from pandas.io.json import json_normalize
import pandas as pd
import json
import numpy as np
import pandas.io.json as pd_json
from sqlalchemy import create_engine


In [2]:
jsonfile = pd.read_json("Resources/mergedjson.json")
jsonfile

modifiedjson = jsonfile.drop(['target_fg','target_og','contributed_by','boil_volume', 'method', 'volume'], axis = 1)

In [3]:
#function to scale srm values based on scale
#https://www.twobeerdudes.com/beer/srm
def srmCat(s):
    if s['srm'] >= 1 and s['srm'] < 3:
        return 'Pale Lager, Pilsner, Witbier, Berliner Weisse'
    elif 3 >= s['srm'] < 4:
        return 'Straw'
    elif 4 >= s['srm'] < 6:
        return 'Pale Gold'
    elif 6 >= s['srm'] < 8:
        return 'Deep Gold'
    elif 8 >= s['srm'] < 10:
        return 'Pale Amber'
    elif 10 >= s['srm'] < 13:
        return 'Medium Amber'
    elif 13 >= s['srm'] < 17:
        return 'Deep Amber'
    elif 17 >= s['srm'] < 20:
        return 'Amber Brown'
    elif 20 >= s['srm'] < 24:
        return 'Brown'
    elif 24 >= s['srm'] < 35:
        return 'Ruby Brown'
    elif 35 >= s['srm'] < 40:
        return 'Deep Brown'
    elif s['srm'] >= 40:
        return 'Black'
    else:
        return 'Undefined'
    
modifiedjson['srm_category'] = modifiedjson.apply(srmCat, axis=1)

In [4]:
#function to scale ibu into 3 categories 
def ibuCat(i):
    if i['ibu'] >= 0 and i['ibu'] < 25:
        return 'Low'
    elif 25 >= i['ibu'] < 38:
        return 'Medium'
    elif i['ibu'] >=40:
        return 'High'
    else:
        return 'Undefined'
    
modifiedjson['ibu_category'] = modifiedjson.apply(ibuCat, axis =1)

In [5]:
#function to scale abv into 3 categories 
def abvCat(a):
    if a['abv'] >= 0 and a['abv'] < 9:
        return 'Low'
    elif 9 >= a['abv'] < 16:
        return 'Medium'
    elif a['abv'] >=16:
        return 'High'
    else:
        return 'Undefined'

modifiedjson['abv_category'] = modifiedjson.apply(abvCat, axis =1)    

In [6]:
#70-82
def attenCat(a):
    if a['attenuation_level'] >=0.0 and a['attenuation_level'] < 70.0:
        return 'Low'
    elif a['attenuation_level'] >= 70.0 and a['attenuation_level'] < 82.0:
        return 'Medium'
    elif a['attenuation_level'] >= 82.0:
        return 'High'
    else:
        'Undefined'
        
modifiedjson['attenuation_category'] = modifiedjson.apply(attenCat, axis =1)    

In [7]:
beerdata = modifiedjson[['id','name','ibu','ibu_category','srm','srm_category','abv','abv_category', 'ebc','attenuation_level','attenuation_category','description',
                        'image_url','ph', 'tagline','brewers_tips']].set_index('id')


In [8]:
with open('Resources/mergedjson.json', encoding='utf8') as data_file:
    data = json.load(data_file)

maltdf = json_normalize(data, meta = ['id'], record_path=['ingredients',['malt']],errors='ignore')
maltdf.drop(columns=['amount'])

maltdf = maltdf.groupby(maltdf['id']).agg(','.join)
maltdf = maltdf.rename(columns={'name': 'malts_name'})
maltdf

hopsdf = json_normalize(data, meta = ['id'], record_path =['ingredients', ['hops']], errors = 'ignore')
hopsdf.drop(columns=['amount'])
hopsdf = hopsdf.groupby(hopsdf['id']).agg(','.join)
hopsdf = hopsdf.rename(columns={'add':'hops_add', 'attribute':'hops_attribute', 'name': 'hops_name'})

fooddf = json_normalize(data, meta =['id'], record_path=['food_pairing'], errors = 'ignore')
fooddf = fooddf.groupby(fooddf['id']).agg(','.join)
fooddf = fooddf.rename(columns = {0:'food_pairing'})

ingredf = maltdf.merge(hopsdf, how ='outer',left_index=True, right_index=True)
ingredf = ingredf.merge(fooddf, how = 'outer', left_index=True, right_index=True)
finaldata = beerdata.merge(ingredf, how= 'outer', left_index=True, right_index=True)

finaldata.to_csv('finalbeerdata.csv')


In [9]:
connection_string = "db/beerdata.sqlite"
engine = create_engine(f'sqlite:///{connection_string}')
finaldata.to_sql(name='finaldata', con=engine, if_exists='append', index=True)